In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# Set style for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
# %%
# Load the dataset
try:
    df = pd.read_csv('owid-covid-data.csv')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")

# %%
# Initial exploration
print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

print("\nColumns:")
print(df.columns.tolist())

print("\nMissing values:")
print(df.isnull().sum().sort_values(ascending=False)[:10])

# %% [markdown]
# ## 2. Data Cleaning

# %%
# Select key countries and columns
countries = ['Kenya', 'United States', 'India', 'Brazil', 'Germany']
cols = ['date', 'location', 'total_cases', 'new_cases', 'total_deaths', 
        'new_deaths', 'people_vaccinated', 'population']

# Filter data
df_clean = df[df['location'].isin(countries)][cols]

# Convert date and sort
df_clean['date'] = pd.to_datetime(df_clean['date'])
df_clean = df_clean.sort_values(['location', 'date'])

# Forward fill missing values for time-series continuity
df_clean = df_clean.groupby('location').apply(lambda x: x.ffill())
# Calculate derived metrics
df_clean['death_rate'] = df_clean['total_deaths'] / df_clean['total_cases']
df_clean['vaccination_rate'] = df_clean['people_vaccinated'] / df_clean['population']

# Drop remaining missing values
df_clean = df_clean.dropna()

# %% [markdown]
# ## 3. Exploratory Data Analysis

# %%
# Basic statistics
print("Global Statistics:")
display(df_clean.describe())

# %%
# Country comparison
country_stats = df_clean.groupby('location').agg({
    'total_cases': 'max',
    'total_deaths': 'max',
    'vaccination_rate': 'max'
}).sort_values('total_cases', ascending=False)

print("\nCountry Comparison:")
display(country_stats)

# %% [markdown]
# ## 4. Data Visualization

# %%
# Line Chart: Cases Over Time
plt.figure(figsize=(14, 7))
for country in countries:
    country_data = df_clean[df_clean['location'] == country]
    plt.plot(country_data['date'], country_data['total_cases'], label=country)

plt.title('COVID-19 Total Cases Over Time')
plt.xlabel('Date')
plt.ylabel('Total Cases (Millions)')
plt.legend()
plt.show()

# %%
# Bar Chart: Total Cases by Country
country_stats['total_cases'].sort_values().plot(kind='barh', color='skyblue')
plt.title('Total COVID-19 Cases by Country')
plt.xlabel('Total Cases (Millions)')
plt.show()

# %%
# Histogram: Daily New Cases
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['new_cases'], bins=30, kde=True)
plt.title('Distribution of Daily New Cases')
plt.xlabel('Daily New Cases')
plt.show()

# %%
# Scatter Plot: Cases vs Deaths
sns.scatterplot(data=df_clean, x='total_cases', y='total_deaths', hue='location')
plt.title('Total Cases vs Total Deaths')
plt.xlabel('Total Cases (Millions)')
plt.ylabel('Total Deaths (Thousands)')
plt.show()

# %%
# Vaccination Progress
plt.figure(figsize=(14, 7))
for country in countries:
    country_data = df_clean[df_clean['location'] == country]
    plt.plot(country_data['date'], country_data['vaccination_rate'], label=country)

plt.title('Vaccination Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Vaccination Rate (%)')
plt.legend()
plt.show()

# %% [markdown]
# ## 5. Choropleth Map (Optional)

# %%
# Prepare data for choropleth
latest_data = df.sort_values('date').groupby('iso_code').last().reset_index()

# Create map
fig = px.choropleth(latest_data,
                    locations="iso_code",
                    color="total_cases",
                    hover_name="location",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Global COVID-19 Case Distribution")
fig.show()



Dataset loaded successfully!
Dataset shape: (350085, 67)

First 5 rows:


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN



Columns:
['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred

,date,total_cases,new_cases,total_deaths,new_deaths,people_vaccinated,population,death_rate,vaccination_rate
count,5042,5.042000e+03,5.042000e+03,5.042000e+03,5042.000000,5.042000e+03,5.042000e+03,5042.000000,5042.000000
mean,2022-06-01 22:18:19.563665152,3.246560e+07,3.742938e+04,4.248724e+05,365.459937,2.465417e+08,4.243833e+08,0.015724,0.552795
min,2020-12-13 00:00:00,1.068010e+05,0.000000e+00,1.866000e+03,0.000000,0.000000e+00,5.402748e+07,0.004437,0.000000
25%,2021-09-23 00:00:00,4.479591e+06,0.000000e+00,9.799800e+04,0.000000,2.233457e+07,8.336984e+07,0.011822,0.263618
50%,2022-06-02 00:00:00,3.272217e+07,4.990000e+02,4.760115e+05,6.000000,1.540881e+08,2.153135e+08,0.016540,0.721675
75%,2023-02-09 00:00:00,4.314954e+07,3.100175e+04,6.670320e+05,317.000000,2.629888e+08,3.382899e+08,0.019047,0.778149
max,2023-10-23 00:00:00,1.034368e+08,1.588891e+06,1.136920e+06,6148.000000,1.027419e+09,1.417173e+09,0.031996,0.880778
std,NaN,2.866072e+07,1.052732e+05,3.464537e+05,778.795584,3.253243e+08,5.078143e+08,0.006376,0.294766


ValueError: 'location' is both an index level and a column label, which is ambiguous.